In [1]:
import pandas as pd
import numpy as np
import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])


PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [2]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/new_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels20.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno

In [6]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm_rm3_for_qe = pt.BatchRetrieve(index, wmodel="DirichletLM")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = dlm_rm3_for_qe >> rm3 >> dlm_rm3_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, 
                                names=['tuned_dlm_rm3_unjudged'],
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics21, qrels, 
                    names=['tuned_dlm_rm3_unjudged20'],
                    eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics20, qrels, 
                names=['tuned_dlm_rm3_unjudged21'],
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))

            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.559156
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7fc9866a01d0> fb_terms=6', '<pyterrier.rewrite.RM3 object at 0x7fc9866a01d0> fb_docs=4', '<pyterrier.rewrite.RM3 object at 0x7fc9866a01d0> fb_lambda=0.2']
Fold 2
Best ndcg_cut_5 is 0.461935
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7fc9866a01d0> fb_terms=4', '<pyterrier.rewrite.RM3 object at 0x7fc9866a01d0> fb_docs=8', '<pyterrier.rewrite.RM3 object at 0x7fc9866a01d0> fb_lambda=0.2']
                     name  ndcg_cut_5  ndcg_cut_25     bpref
0  tuned_dlm_rm3_unjudged    0.651599     0.556775  0.492538
                       name  ndcg_cut_5  ndcg_cut_25     bpref
0  tuned_dlm_rm3_unjudged20    0.623817     0.587539  0.572419
                       name  ndcg_cut_5  ndcg_cut_25     bpref
0  tuned_dlm_rm3_unjudged21    0.679949     0.525383  0.411027


In [5]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm_rm3_for_qe = pt.BatchRetrieve(index, wmodel="DirichletLM")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = dlm_rm3_for_qe >> rm3 >> dlm_rm3_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics21, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))

            i += 1
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.518446
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7fc987ca0fd0> fb_terms=12', '<pyterrier.rewrite.RM3 object at 0x7fc987ca0fd0> fb_docs=6', '<pyterrier.rewrite.RM3 object at 0x7fc987ca0fd0> fb_lambda=0.2']
Fold 2
Best ndcg_cut_5 is 0.384881
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7fc987ca0fd0> fb_terms=8', '<pyterrier.rewrite.RM3 object at 0x7fc987ca0fd0> fb_docs=6', '<pyterrier.rewrite.RM3 object at 0x7fc987ca0fd0> fb_lambda=0.2']
                                 name  ndcg_cut_5  ndcg_cut_25    bpref
0  conclusions_tuned_dlm_rm3_unjudged    0.665271     0.611759  0.58175
                                   name  ndcg_cut_5  ndcg_cut_25     bpref
0  conclusions_tuned_dlm_rm3_unjudged20     0.67535     0.559794  0.453295
                                   name  ndcg_cut_5  ndcg_cut_25     bpref
0  conclusions_tuned_dlm_rm3_un